### 1. Setup inicial do Spark e Postgres

#### 1.1 Spark

In [3]:
import os
from dotenv import load_dotenv
import warnings, logging
from pyspark.sql import SparkSession


# Silenciar warnings/logs
import warnings, logging
warnings.filterwarnings("ignore")

for name in ["py4j", "py4j.java_gateway", "botocore", "boto3", "urllib3"]:
    logging.getLogger(name).setLevel(logging.ERROR)
print("🔕 Warnings e logs reduzidos.")


# carregar variáveis do .env
load_dotenv()

# Parar qualquer sessão Spark existente
try:
    if 'spark' in locals():
        spark.stop()
        print("Spark session anterior fechada")
    else:
        print("Nenhuma sessão anterior encontrada")
except Exception as e:
    print(f"Erro ao fechar sessão Spark: {e}")

spark = (
    SparkSession.builder
      .appName("IBGE_Gold")
      .config("spark.driver.memory", "4g")
      .config("spark.executor.memory", "4g")
      .config("spark.driver.bindAddress", "127.0.0.1")
      .config("spark.driver.host", "localhost")
      .config("spark.jars.packages", "org.postgresql:postgresql:42.7.3")
      .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")

print("✅ Spark session criada.")

🔕 Warnings e logs reduzidos.
Nenhuma sessão anterior encontrada


:: loading settings :: url = jar:file:/opt/homebrew/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/hgirardi/.ivy2.5.2/cache
The jars for the packages stored in: /Users/hgirardi/.ivy2.5.2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-05eca980-dcf8-4060-bcf1-9d37a743f903;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 56ms :: artifacts dl 2ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------------------

✅ Spark session criada.


#### 1.2 Postgres

In [4]:
# Dados de conexão com o Postgres para usar com o Spark
jdbc_url = f"jdbc:postgresql://{os.getenv('POSTGRES_HOST')}:{os.getenv('POSTGRES_PORT')}/{os.getenv('POSTGRES_DATABASE')}?reWriteBatchedInserts=true"
connection_properties = {
    "user": os.getenv('POSTGRES_USER'),
    "password": os.getenv('POSTGRES_PASSWORD'),
    "driver": "org.postgresql.Driver",
    "batchsize": "5000"
}

### 2. Carregar Silver

In [ ]:
PARTES = 12             # paralelismo na leitura do JDBC
FETCHSIZE  = "50000"    # tamanho do lote JDBC. Quanto maior, menos idas e vindas ao banco.

query = f"(SELECT *, ntile({PARTES}) OVER (ORDER BY ctid) AS __p FROM silver.covid) t"

df_silver = (spark.read.format("jdbc")
    .option("url", jdbc_url)
    .option("dbtable", query)
    .option("partitionColumn", "__p")
    .option("lowerBound", "1")
    .option("upperBound", str(PARTES))
    .option("numPartitions", str(PARTES))
    .option("fetchsize", FETCHSIZE)
    .options(**connection_properties)
    .load()
    .drop("__p")
)

df_silver.createOrReplaceTempView("silver_covid")
spark.sql("CACHE TABLE silver_covid")  # acelera as próximas queries
_ = spark.table("silver_covid").limit(1).count()

DataFrame[]

In [7]:
_ = spark.table("silver_covid").limit(1).count()